# 1. Formulação do problema

    Variáveis de Decisão:
        ti​ - Tempo de pouso do avião i (variável contínua);
        xij​ - Variável binária que vale 1 se o avião i pousar antes do avião j, 0 caso contrário;
        ei - Adiantamento do tempo ideal de pouso para o avião i (ei ≥ 0);
        di - Atraso do tempo ideal de pouso para o avião i (di ≥ 0).

    Parâmetros:
    
        Ri - tempo de sua detecção pelo radar;
        Ei - tempo inicial de pouso;
        Ti - tempo ideal para o pouso;
        Li - tempo final que o avião i pode pousar;
        
    Função Objetivo:

        Minimizar a penalidade total de pousos fora do tempo ideal:

## **Minimizar ∑ (gi * ei + hi * di)**

    Onde:

    gi​ e hi​ são as penalidades por pousar antes ou depois do tempo ideal, respectivamente.

    S.a:

        Restrições de Separação:
            O intervalo entre os pousos deve ser suficiente para garantir segurança e garantir uma sequência válida de aterrissagens:

              tj ≥ ti + sij − M(1 − xij), ∀i,j, i≠j

              Se xij = 1, então tj ≥ ti + sij, garantindo a separação mínima entre os pousos
              Se xij = 0, a restrição é relaxada devido ao termo M(1 − xij)

              ti ≥ tj + sji − Mxij, ∀i,j, i≠j

              Similar à restrição anterior, mas garantindo que a relação entre ti​ e tj​ seja consistente dependendo do valor de xij​.

        Restrições de Sequenciamento:
            Garante que para cada par de aviões (i,j)(i,j), um deve pousar antes do outro, evitando ciclos:

              xij + xji = 1, ∀i,j, i≠j

              Isso assegura que se o avião ii pousar antes de jj, então xij=1 e xji=0, caso contrário, xij=0 e xji=1
        
        Variáveis Binárias:

                xij​ + xji ​= 1 ∀i, j, i≠j

        Cálculo de adiantamento e atraso:

                ei ​≥ Ti ​− ti ​∀i
                di ≥ ti − Ti ∀i
                ei ≤ Ti − Ei ∀i
                di ≤ Li − Ti ∀i
                



# Resolvendo usando o solver GLPK

In [2]:
!pip install pulp
!sudo apt-get install glpk-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 85.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 18 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0

In [12]:
import time
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpBinary, PULP_CBC_CMD, value

def ler_dados_instancia(caminho_arquivo):
    with open(caminho_arquivo, 'r') as arquivo:
        linha_inicial = arquivo.readline().strip().split()
        qtdeDeAvioes = int(linha_inicial[0])

        tempos = []
        separacoes = []
        penalidades = []

        for _ in range(qtdeDeAvioes):
            linha_tempos = arquivo.readline().strip().split()
            while len(linha_tempos) < 6:
                linha_tempos.extend(arquivo.readline().strip().split())
            ri, ei, ti, li, gi, hi = map(float, linha_tempos[:6])
            tempos.append((ri, ei, ti, li, gi, hi))
            penalidades.append((gi, hi))

            linha_separacao = []
            while len(linha_separacao) < qtdeDeAvioes:
                linha_separacao.extend(arquivo.readline().strip().split())
            separacoes.append(list(map(float, linha_separacao[:qtdeDeAvioes])))

        return qtdeDeAvioes, tempos, separacoes, penalidades


def resolver_modelo_glpk(qtdeDeAvioes, tempos, separacoes, penalidades):
    modelo = LpProblem("SequenciamentoPousos", LpMinimize)

    # Variáveis de decisão
    t = [LpVariable(f"t_{i}", lowBound=tempos[i][1], upBound=tempos[i][3], cat='Continuous') for i in range(qtdeDeAvioes)]
    e = [LpVariable(f"e_{i}", lowBound=0, cat='Continuous') for i in range(qtdeDeAvioes)]
    d = [LpVariable(f"d_{i}", lowBound=0, cat='Continuous') for i in range(qtdeDeAvioes)]
    x = [[LpVariable(f"x_{i}_{j}", cat=LpBinary) if i != j else None for j in range(qtdeDeAvioes)] for i in range(qtdeDeAvioes)]

    # Função objetivo
    modelo += lpSum(penalidades[i][0] * e[i] + penalidades[i][1] * d[i] for i in range(qtdeDeAvioes)), "PenalidadeTotal"

    # Constante grande M
    M = max(tempos[i][3] for i in range(qtdeDeAvioes)) + max(max(separacoes[i]) for i in range(qtdeDeAvioes))

    # Restrições de separação e sequenciamento
    for i in range(qtdeDeAvioes):
        for j in range(qtdeDeAvioes):
            if i != j:
                modelo += t[j] >= t[i] + separacoes[i][j] - M * (1 - x[i][j]), f"Separacao_{i}_{j}_A"
                modelo += t[i] >= t[j] + separacoes[j][i] - M * x[i][j], f"Separacao_{j}_{i}_B"
                modelo += x[i][j] + x[j][i] == 1, f"Sequencia_{i}_{j}"

    # Cálculo de adiantamento e atraso
    for i in range(qtdeDeAvioes):
        modelo += e[i] >= tempos[i][2] - t[i], f"Adiantamento_{i}"
        modelo += d[i] >= t[i] - tempos[i][2], f"Atraso_{i}"
        modelo += e[i] <= tempos[i][2] - tempos[i][1], f"LimiteAdiantamento_{i}"
        modelo += d[i] <= tempos[i][3] - tempos[i][2], f"LimiteAtraso_{i}"

    # Resolver o modelo com tempo limite
    inicio_tempo = time.time()
    resultado = modelo.solve(PULP_CBC_CMD(msg=True, timeLimit=640))
    tempo_execucao = time.time() - inicio_tempo

    if modelo.status == 1:  # Solução ótima encontrada
        tempos_pouso = [value(t[i]) for i in range(qtdeDeAvioes)]
        valor_otimo = value(modelo.objective)
        print(f"Valor ótimo da solução: {valor_otimo}")
        print(f"Tempos de pouso: {', '.join(map(str, tempos_pouso))}")
        print(f"Tempo de execução do Solver: {tempo_execucao:.2f} segundos")
    else:  # Solução aproximada
        tempos_pouso = [value(t[i]) for i in range(qtdeDeAvioes) if t[i].varValue is not None]
        valor_aproximado = value(modelo.objective) if value(modelo.objective) is not None else float('inf')
        print("Solução aproximada encontrada (não ótima dentro do tempo limite)")
        print(f"Penalidade aproximada: {valor_aproximado}")
        print(f"Tempos de pouso aproximados: {', '.join(map(str, tempos_pouso))}")
        print(f"Tempo de execução do Solver: {tempo_execucao:.2f} segundos")


def executarInstancia():
    for i in range(1, 9):
        caminho_arquivo = f"problema_do_aviao/instances/0{i}.dat"
        qtdeDeAvioes, tempos, separacoes, penalidades = ler_dados_instancia(caminho_arquivo)
        resolver_modelo_glpk(qtdeDeAvioes, tempos, separacoes, penalidades)


executarInstancia()


Valor ótimo da solução: 699.999999999892
Tempos de pouso: 165.0, 258.0, 98.0, 106.0, 118.0, 126.0, 134.0, 142.0, 150.0, 180.0
Tempo de execução do Solver: 0.22 segundos
Valor ótimo da solução: 1480.0
Tempos de pouso: 196.0, 250.0, 90.0, 98.0, 106.0, 130.0, 122.0, 114.0, 138.0, 151.0, 341.0, 313.0, 181.0, 171.0, 344.0
Tempo de execução do Solver: 1.69 segundos
Valor ótimo da solução: 820.0
Tempos de pouso: 82.0, 197.0, 184.0, 117.0, 261.0, 101.0, 229.0, 109.0, 133.0, 141.0, 149.0, 125.0, 336.0, 316.0, 258.0, 409.0, 339.0, 287.0, 160.0, 169.0
Tempo de execução do Solver: 0.49 segundos
Valor ótimo da solução: 2520.0
Tempos de pouso: 82.0, 90.0, 201.0, 270.0, 98.0, 122.0, 130.0, 114.0, 106.0, 280.0, 295.0, 146.0, 138.0, 291.0, 186.0, 154.0, 170.0, 178.0, 162.0, 357.0
Tempo de execução do Solver: 21.56 segundos
Valor ótimo da solução: 3100.0
Tempos de pouso: 201.0, 246.0, 82.0, 90.0, 98.0, 122.0, 114.0, 106.0, 130.0, 138.0, 307.0, 280.0, 170.0, 146.0, 301.0, 393.0, 162.0, 178.0, 154.0, 186.